In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
ss = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

# Add S3 paths 

In [ ]:
mfcc = ss.read.csv('mfcc_final.csv', header=True)
cqt = ss.read.csv('/Users/RusL/Desktop/audio-project/cqt_final.csv', header=True)
gender = ss.read.csv('Lab41-SRI-VOiCES-speaker-gender-dataset.csv', header=True)

In [ ]:
cqt.cache()
gender.cache()
mfcc.cache()

# Use the cell below to drop useless columns

For example, there might be 'Unnamed: 0'

In [ ]:
cqt.columns # to see what columns are there
cqt = cqt.drop(...)
# Repeat this with mfcc and gender

# Process the filename and get the speaker id

In [ ]:
# define a udf that extracts the speaker id out of the filename
def get_speaker(filename):
    i = filename.index('sp')
    return int(filename[i+2:i+6])
speaker = udf(get_speaker)

In [ ]:
# process speaker id
cqt = cqt.withColumn('Speaker', speaker('FileName')).drop('FileName')
mfcc = mfcc.withColumn('Speaker', speaker('FileName')).drop('FileName')

# Join the tables and check the result

In [ ]:
# join the dataframe
results = gender.join(mfcc, on='Speaker', how='rightOuter')\
                .join(cqt, on='Speaker', how='inner')

In [ ]:
results.show()

In [ ]:
# check the number of observations
print(results.count())
print(cqt.count())

In [ ]:
sc.stop()